In [ ]:
!apt-get update &&    apt-get install -y   software-properties-common &&  add-apt-repository ppa:deadsnakes/ppa -y 
!apt-get update 

!apt-get install -y    python3.10    python3.10-venv    python3.10-distutils    python3-pip    wget    git    libgl1    libreoffice    fonts-noto-cjk    fonts-wqy-zenhei    fonts-wqy-microhei    ttf-mscorefonts-installer    fontconfig    libglib2.0-0    libxrender1    libsm6    libxext6    poppler-utils \

!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

In [ ]:
!pip install -U "magic-pdf[full]"
!pip install flask pyngrok
!pip install huggingface_hub

In [37]:
!wget https://github.com/opendatalab/MinerU/raw/master/scripts/download_models_hf.py -O download_models_hf.py
!python download_models_hf.py

--2025-06-08 07:10:31--  https://github.com/opendatalab/MinerU/raw/master/scripts/download_models_hf.py
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opendatalab/MinerU/master/scripts/download_models_hf.py [following]
--2025-06-08 07:10:31--  https://raw.githubusercontent.com/opendatalab/MinerU/master/scripts/download_models_hf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2386 (2.3K) [text/plain]
Saving to: ‘download_models_hf.py’

download_models_hf. 100%[===================>]   2.33K  --.-KB/s    in 0s      

2025-06-08 07:10:31 (35.2 MB/s) - ‘download_models_hf.py’ saved [2386/2386]



In [ ]:
!wget https://github.com/opendatalab/MinerU/raw/master/magic-pdf.template.json &&  cp magic-pdf.template.json /root/magic-pdf.json
!sed -i 's|cpu|cuda|g' /root/magic-pdf.json
!cat /root/magic-pdf.json

In [ ]:
!rm $(ls ./ ) -r


import os
from os.path import sep
import fitz
from PIL import Image
from flask import Flask, request, jsonify ,send_file,Response
from zipfile import ZipFile
import json
from io import  BytesIO
from pyngrok import ngrok
import traceback
from kaggle_secrets import UserSecretsClient

from magic_pdf.data.data_reader_writer import (
    FileBasedDataWriter,
)
from magic_pdf.data.dataset import ImageDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze,batch_doc_analyze
from magic_pdf.operators.models import InferenceResult, PipeResult


# --- 1. SETUP THE TUNNEL ---
# Authenticate ngrok using the secret you stored
user_secrets = UserSecretsClient()
ngrok.set_auth_token(user_secrets.get_secret("NGROK_AUTH_TOKEN"))

# Open a tunnel to the Flask app port (we'll use 5000)
public_url = ngrok.connect(5000)
print(f"✅ Kaggle is now live at: {public_url}")


# --- 2. LOAD YOUR HEAVY MODEL (DO THIS ONLY ONCE) ---
# This is where you would load your model, weights, etc.
# Example:
# from your_module import load_model, predict_function
# model = load_model('/kaggle/input/my-repo-name/models/best_model.pth')
print("✅ Model loaded successfully (this is a placeholder).")

def detect_layout_miner_u_online(img_bytes: bytes, data: dict):


    exam = data.get("exam", "")
    d_mode = data.get("display-mode", "")
    nr = "nr" + str(data.get("number", 0)) + "_"
    want = data.get("want", "md_content.md")

    key = exam + d_mode
    f_dir = sep.join([".","results",key])
    os.makedirs(f_dir, exist_ok=True)
    f_path = f"{f_dir}{sep}{nr}{want}"

    md_image_dir  = f"{f_dir}{sep}{nr}images"
    if not os.path.exists(f_path):
        print("ocring ....")
        image_dir = str(os.path.basename(md_image_dir))

        os.makedirs(md_image_dir, exist_ok=True)

        image_writer, md_writer = FileBasedDataWriter(
            md_image_dir
        ), FileBasedDataWriter(f_dir)

        lang = "ch_server"
        ds = ImageDataset(img_bytes, lang=lang)

        inf_res: InferenceResult = ds.apply(
            doc_analyze,
            ocr=True,
            lang=lang,
            show_log=True,
        )

        p5 = f"{f_dir}{sep}{nr}draw5.png"
        inf_res.draw_model(p5)
        pdf_to_png(p5)

        pip_res: PipeResult = inf_res.pipe_ocr_mode(image_writer, lang=lang)


        pip_res.dump_md(md_writer, f"{nr}md_content.md", image_dir)

        p2 = f"{f_dir}{sep}{nr}draw2.png"
        pip_res.draw_layout(p2)
        pdf_to_png(p2)

        p3 = f"{f_dir}{sep}{nr}draw3.png"
        pip_res.draw_span(p3)
        pdf_to_png(p3)

        p4 = f"{f_dir}{sep}{nr}draw4.png"
        pip_res.draw_line_sort(p4)
        pdf_to_png(p4)
    else:
        print("using cached version")

    if want != "md_content.md":
        return send_file(f_path, as_attachment=True)

    file_paths = [f"{md_image_dir}{sep}{f}" for f in os.listdir(md_image_dir)]
    file_paths.append(f_path)
    memory_file = BytesIO()
    with ZipFile(memory_file, 'w') as zf:
        for file_path in file_paths:
            zf.write(file_path, os.path.basename(file_path))
    memory_file.seek(0)
    return Response(memory_file.getvalue(), mimetype='application/zip',
                     headers={'Content-Disposition': f'attachment;filename=md_content.zip'}
                    )


def pdf_to_png(pdf_path):

    dpi = 150
    doc = fitz.open(pdf_path)
    page = doc.load_page(0)
    pix = page.get_pixmap(dpi=dpi)
    mode = "RGBA" if pix.alpha else "RGB"
    img = Image.frombytes(mode, (pix.width, pix.height), pix.samples)
    doc.close()
    img.save(pdf_path, "png")



# --- 3. CREATE THE FLASK APP ---
app = Flask(__name__)
@app.route("/", methods=["GET"])
def say_hallo():
    return jsonify({"message":"hallo zakir"})
    

@app.route("/predict", methods=["POST"])
def predict():
    """The main endpoint for your GUI to call."""
    try:
        image_file = request.files['image']
        image_bytes = image_file.read()
        print(request.form)
        return  detect_layout_miner_u_online(image_bytes,request.form)


    except Exception as e:
        print(traceback.format_exc())
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

@app.route("/predict/advance", methods=["POST"])
def predict_advance():
    """The main endpoint for your GUI to call."""
    try:
        image_file = request.files['image']
        image_group_bytes :bytes = image_file.read()
        fo = request.form
        fo = json.loads(fo.get("json"))
        print(fo)
        sep = fo.get("seperator").encode("latin")
        idx = fo.get("idx")
        data_sets = []
        lang = "ch_server"
        for im_bytes in image_group_bytes.split(sep):
            if  im_bytes:
                data_sets.append( ImageDataset(im_bytes, lang=lang))

        inf_res_list : list[InferenceResult] = batch_doc_analyze(
            data_sets,
            parse_method="ocr" ,
            lang=lang,
            show_log=True,
        )
        image_dir = "./temp"
        os.makedirs(image_dir,exist_ok=True)
        image_writer = FileBasedDataWriter( image_dir)
        final_res = {}

        final_res = {"md-content":{},"content-list" : {},"middle-json": {}}
        for i,inf_res in enumerate(inf_res_list):
            pip_res: PipeResult = inf_res.pipe_ocr_mode(image_writer,lang=lang)
            cont = pip_res.get_content_list(image_dir)
            midd_json = pip_res.get_middle_json()
            md_cont = pip_res.get_markdown(image_dir)
            id = idx[i]
            final_res["md-content"][id] = md_cont
            final_res["content-list"][id] = cont
            final_res["middle-json"][id] = midd_json

        return jsonify(final_res) , 200


    except Exception as e:
        print(traceback.format_exc())
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

# This will run the Flask app and keep the notebook cell running.
app.run(port=5000)

rm: missing operand
Try 'rm --help' for more information.
✅ Kaggle is now live at: NgrokTunnel: "https://f152-34-127-104-190.ngrok-free.app" -> "http://localhost:5000"
✅ Model loaded successfully (this is a placeholder).
 * Serving Flask app '__main__'
 * Debug mode: off


2025-06-08 07:26:26.485 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.509 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.532 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.543 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.562 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.579 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.592 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:26:26.603 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server


{'idx': ['0580_m23_qp_32_2', '0580_m23_qp_32_2_a_i', '0580_m23_qp_32_2_a_ii', '0580_m23_qp_32_2_a_iii', '0580_m23_qp_32_2_b_i', '0580_m23_qp_32_2_b_ii', '0580_m23_qp_32_2_b_iii', '0580_m23_qp_32_2_b_iv'], 'seperator': 'IAM_A_SEPERATOR_PLEASE'}


2025-06-08 07:26:26.774 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:batch_doc_analyze:223 - Batch 1/1: 8 pages/8 pages
2025-06-08 07:26:26.776 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:278 - gpu_memory: 16 GB, batch_ratio: 16
MFD Predict: 100%|██████████| 8/8 [00:01<00:00,  7.88it/s]
MFR Predict: 0it [00:00, ?it/s]
Processing pages: 100%|██████████| 1/1 [00:00<00:00, 18.41it/s]
2025-06-08 07:30:42.547 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:30:42.574 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:30:42.593 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:30:42.606 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:30:42.615 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:30:42.637 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 07:30:42.6

{'idx': ['0580_m23_qp_32_1', '0580_m23_qp_32_1_a', '0580_m23_qp_32_1_b_i', '0580_m23_qp_32_1_b_ii', '0580_m23_qp_32_1_c', '0580_m23_qp_32_1_d_i', '0580_m23_qp_32_1_d_ii'], 'seperator': 'IAM_A_SEPERATOR_PLEASE'}


...

2025-06-08 08:45:50.095 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:45:50.222 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:45:50.327 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:45:50.466 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:45:50.687 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:45:52.958 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:batch_doc_analyze:223 - Batch 1/1: 7 pages/7 pages
2025-06-08 08:45:52.960 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:278 - gpu_memory: 16 GB, batch_ratio: 16
Processing pages: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]
2025-06-08 08:58:17.693 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:58:17.745 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:58:17.7

{'idx': ['0580_m23_qp_32_1_a', '0580_m23_qp_32_1_b_i', '0580_m23_qp_32_1_b_ii', '0580_m23_qp_32_1_c', '0580_m23_qp_32_1_d', '0580_m23_qp_32_1_d_i', '0580_m23_qp_32_1_d_ii'], 'seperator': 'IAM_A_SEPERATOR_PLEASE'}


2025-06-08 08:58:17.917 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:58:17.984 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:58:18.430 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:batch_doc_analyze:223 - Batch 1/1: 7 pages/7 pages
2025-06-08 08:58:18.431 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:278 - gpu_memory: 16 GB, batch_ratio: 16
Processing pages: 100%|██████████| 1/1 [00:00<00:00,  4.90it/s]
2025-06-08 08:59:32.137 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:59:32.142 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:59:32.156 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:59:32.164 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:59:32.175 | INFO     | magic_pdf.data.dataset:__init__:256 - lang: ch_server
2025-06-08 08:59:32.1

{'idx': ['0580_m23_qp_32_2_a', '0580_m23_qp_32_2_a_i', '0580_m23_qp_32_2_a_ii', '0580_m23_qp_32_2_a_iii', '0580_m23_qp_32_2_b', '0580_m23_qp_32_2_b_i', '0580_m23_qp_32_2_b_ii', '0580_m23_qp_32_2_b_iii', '0580_m23_qp_32_2_b_iv'], 'seperator': 'IAM_A_SEPERATOR_PLEASE'}


MFD Predict: 100%|██████████| 9/9 [00:00<00:00, 12.52it/s]
MFR Predict: 0it [00:00, ?it/s]
Processing pages: 100%|██████████| 1/1 [00:00<00:00, 18.43it/s]
